**ENSEMBLE LEARNING.**

Technique createsmultiple models and then combines them to produce better results than any of the single models

**Random Forest**
        - Can be used for classification and regression
        - Easily handles outlier
        - Accepts various types of inputs(Continuos, ordinal)
        - Less likely to overfit
        - Outputs feature importances.

In [62]:
import string 
import pandas as pd
import nltk
from nltk.tokenize import wordpunct_tokenize,word_tokenize
import os
import re

dataset = pd.read_csv(os.getcwd()+'\\Ex_Files_NLP_Python_ML_EssT\\Exercise Files\\Ch01\\01_03\\Start\\SMSSpamCollection.tsv', header=None, sep='\t')
dataset.columns = ['label', 'body_text']
dataset.sample(5)

#create new features containing the percentage of punctuation in text and the body length of text 

def count_punctuation_and_body_len(text):
    punct_count = [char for char in text if char in string.punctuation]
    body_len = len(text) - text.count(" ")
    punct_perc = round(len(punct_count)/body_len, 3)*100
    return body_len, punct_perc

sp_words = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
def clean_data(text):
    tokens = ''.join([char for char in text if char not in string.punctuation])
    tokens_punct = re.split('\s+', tokens)
    tokens_stop = [token.lower()  for token in tokens_punct if token not in sp_words]
    clean_text = [ps.stem(word) for word in tokens_stop]
    return clean_text

dataset['cleaned_text'] = dataset['body_text'].apply(lambda x: clean_data(x))
dataset['body_len'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[0])
dataset['punc%'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[1])    
dataset.head()

,label,body_text,cleaned_text,body_len,punc%
0,ham,I've been searching for the right words to tha...,"[ive, search, right, word, thank, breather, i,...",160,2.5
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",128,4.7
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, dont, think, goe, usf, live, around, ...",49,4.1
3,ham,Even my brother is not like to speak with me. ...,"[even, brother, like, speak, they, treat, like...",62,3.2
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[i, have, a, date, on, sunday, with, will]",28,7.1


In [73]:
#vectorizing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def return_cleaned_text_string(text):
    string_version = ''.join(text)
    return string_version

dataset['body_text_new'] = dataset['cleaned_text'].apply(lambda x: return_cleaned_text_string(x))

#tfidf
tfidf_vect = TfidfVectorizer(analyzer=clean_data)
x_tfidf = tfidf_vect.fit_transform(dataset['body_text'])
x_tfidf_features = pd.concat([dataset[['body_len','punc%']], pd.DataFrame(x_tfidf.toarray())], axis=1)
#count
count_vect = CountVectorizer(analyzer=clean_data)
x_count = count_vect.fit_transform(dataset['body_text'])
x_count_features = pd.concat([dataset[['body_len','punc%']], pd.DataFrame(x_tfidf.toarray())], axis=1)

x_count_features.head()

,body_len,punc%,0,1,2,3,4,5,6,7,...,8265,8266,8267,8268,8269,8270,8271,8272,8273,8274
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
from sklearn.ensemble import RandomForestClassifier

print (dir(RandomForestClassifier))
print (RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_make_estimator', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,

In [ ]:
RUNNING RANDOM FOREST 

In [65]:
#randomforest in action
from sklearn.model_selection import KFold, cross_val_score

rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, x_features, dataset['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\Lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py

In [35]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [66]:
X_train, X_test, y_train, y_test = train_test_split(x_features, dataset['label'], test_size=0.2)
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[:10]

[(0.050720577465307733, 1791),
 (0.04486622923962251, 'body_len'),
 (0.033798426316406512, 3146),
 (0.032688908167355014, 6350),
 (0.032083499222619966, 4842),
 (0.024182704876344933, 2025),
 (0.020846811909829639, 7098),
 (0.01876435512718623, 7259),
 (0.015553156436916302, 6051),
 (0.015257888874461292, 7433)]

In [67]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
print ('Precision: {} / Recall: {} / fscore: {} / Accuracy: {}'.format(round(precision, 3),
                                                                      round(recall, 3),
                                                                      round(fscore, 3),
                                                                      round((y_pred == y_test).sum()/len(y_pred), 3)))

Precision: 1.0 / Recall: 0.617 / fscore: 0.763 / Accuracy: 0.956


In [68]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} --- Precision: {} / Recall: {} / Accuracy: {}'.format(n_est, depth,
                                                                              round(precision,3), 
                                                                              round(recall, 3),
                                                                            round((y_pred == y_test).sum()/len(y_pred), 3)))
    

In [69]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 --- Precision: 1.0 / Recall: 0.25 / Accuracy: 0.914
Est: 10 / Depth: 20 --- Precision: 1.0 / Recall: 0.625 / Accuracy: 0.957
Est: 10 / Depth: 30 --- Precision: 1.0 / Recall: 0.75 / Accuracy: 0.971
Est: 10 / Depth: None --- Precision: 0.99 / Recall: 0.805 / Accuracy: 0.977
Est: 50 / Depth: 10 --- Precision: 1.0 / Recall: 0.25 / Accuracy: 0.914
Est: 50 / Depth: 20 --- Precision: 1.0 / Recall: 0.664 / Accuracy: 0.961
Est: 50 / Depth: 30 --- Precision: 1.0 / Recall: 0.742 / Accuracy: 0.97
Est: 50 / Depth: None --- Precision: 1.0 / Recall: 0.836 / Accuracy: 0.981
Est: 100 / Depth: 10 --- Precision: 1.0 / Recall: 0.297 / Accuracy: 0.919
Est: 100 / Depth: 20 --- Precision: 1.0 / Recall: 0.617 / Accuracy: 0.956
Est: 100 / Depth: 30 --- Precision: 1.0 / Recall: 0.758 / Accuracy: 0.972
Est: 100 / Depth: None --- Precision: 1.0 / Recall: 0.867 / Accuracy: 0.985


In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [75]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300], 'max_depth':[30, 60, 90, None]}
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(x_tfidf_features, dataset['label'])
try:
    pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]
except Exception as e:
    print(e)

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\Lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py

In [76]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 50, 300], 'max_depth':[30, 60, 90, None]}
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(x_count_features, dataset['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\Lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py